In [26]:
INPUTS = ['production/puri/input', 'production/puri/modeloutput/simplified']
OUTPUT = 'production/puri/modeloutput/rooftype'
REGION = 'puri'
MODELNAME = 'inceptionv3'
CHECKPOINT = 'rooftypeclassification'
SIZE = 299

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import argparse
import rasterio as rio
from rasterio.mask import mask

import geopandas as gpd
import geojson

import numpy as np
import torch
import torch.nn as nn
import torch.distributed as dist


from skimage import io
from scipy import ndimage as ndi
import cv2

from tqdm import tqdm
from pathlib import Path
import os
import sys 

sys.path.append('nn/')
from classifier import Classifier
from azureml.core.model import Model

In [8]:
CHECKPOINT: 'checkpoints/rooftypeclassification'

In [9]:
model = Classifier(name=MODELNAME, num_classes=7)
model = nn.DataParallel(model)

# if torch.cuda.is_available():
#     model = model.cuda()
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


#load model checkpoint
best_score = 0
start_epoch = 0
#checkpoint_path = Path(Model.get_model_path(CHECKPOINT, version=2))
checkpoint_path = Path('checkpoints/rooftypeclassification')
if checkpoint_path.exists():
    print(f'Loading Checkpoint from {CHECKPOINT}')
    checkpoint = torch.load(str(checkpoint_path), map_location='cpu')
    loaded_dict = checkpoint['state_dict']
    state_dict = model.state_dict() 
    for key in state_dict:
        if key in loaded_dict:
            state_dict[key] = loaded_dict[key]

    model.load_state_dict(state_dict)
    start_epoch = checkpoint['epoch']
    best_score = checkpoint['best_score']

    print(f'Loaded Checkpoint from: {CHECKPOINT}, \
                with epoch {start_epoch} and best score {best_score}')

Loading Checkpoint from rooftypeclassification
Loaded Checkpoint from: rooftypeclassification,                 with epoch 92 and best score 80.36363636363636


In [14]:
RASTER = Path(INPUTS[0])
VECTOR = Path(INPUTS[1])
#OUTPUT = Path(OUTPUT)

# Read Simplified Footrprints

In [15]:
footprints_file = f'{VECTOR}/{REGION}_simplified_footprints_from_model.geojson'
print(f'Reading simplified footprints from {footprints_file}')
footprints = gpd.read_file(f'{footprints_file}').to_crs(4326)

Reading simplified footprints from production/puri/modeloutput/simplified/puri_simplified_footprints_from_model.geojson


In [16]:
footprints.head()

,geometry
0,"POLYGON ((85.80406 19.79352, 85.80406 19.79352..."
1,"POLYGON ((85.80194 19.79276, 85.80195 19.79276..."
2,"POLYGON ((85.80227 19.79280, 85.80227 19.79280..."
3,"POLYGON ((85.80213 19.79275, 85.80214 19.79275..."
4,"POLYGON ((85.80251 19.79264, 85.80251 19.79261..."


In [17]:
footprints.info

<bound method DataFrame.info of                                                 geometry
0      POLYGON ((85.80406 19.79352, 85.80406 19.79352...
1      POLYGON ((85.80194 19.79276, 85.80195 19.79276...
2      POLYGON ((85.80227 19.79280, 85.80227 19.79280...
3      POLYGON ((85.80213 19.79275, 85.80214 19.79275...
4      POLYGON ((85.80251 19.79264, 85.80251 19.79261...
...                                                  ...
23520  POLYGON ((85.82890 19.81843, 85.82891 19.81843...
23521  POLYGON ((85.82923 19.81846, 85.82923 19.81846...
23522  POLYGON ((85.82948 19.81846, 85.82948 19.81846...
23523  POLYGON ((85.82897 19.81843, 85.82897 19.81843...
23524  POLYGON ((85.82910 19.81843, 85.82910 19.81843...

[23525 rows x 1 columns]>

# Roof Type Prediction

In [18]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#mapping the model output to rooft type id defined 
#for ex in model RCC is predicted as 3 but RCC id defined is 1(in phase 1)
dt_mapping = {
    0 : '2' ,
    1 : '3',
    2 : '4',
    3 : '1',
    4 : '7',
    5 : '5',
    6 : '6'
}

In [19]:
print('Predicting roof type')
model.eval()
building_dwell_type = []
# for all the input files
for raster_file in RASTER.glob('*.tif'):
    with rio.open(raster_file) as rf:
        for idx, footprint in footprints.iterrows():
            try:
                #crop the building from source image
                building, building_transform = mask(rf, shapes=[footprint.geometry], 
                                                    crop=True)
                _building = building.copy()
                
                #resize and normalise the data
                _building = np.transpose(_building, (1, 2, 0))
                _building = cv2.resize(_building, (SIZE, SIZE), 
                                           interpolation=cv2.INTER_NEAREST)
                _building = np.asarray(_building, dtype='float32') / 255
                for i in range(3):
                    _building[..., i] = (_building[..., i] - mean[i]) / std[i]
                _building = torch.from_numpy(_building.transpose((2, 0, 1)).copy()).float()
                _building = _building.unsqueeze(0)
                
                #run prediction
                with torch.no_grad():
                    output = model(_building).cpu().detach().numpy()
                    label = np.argmax(output)
                    dwell_type = dt_mapping[label]

                    building_dwell_type.append({
                        'geometry': footprint.geometry,
                        'label': dwell_type
                    })
                    
            except Exception as e:
                continue
                
            

Predicting roof type


# Check Results

In [20]:
len(building_dwell_type)

24947

In [21]:
bdt_gdf = gpd.GeoDataFrame(building_dwell_type)

In [22]:
bdt_gdf.head()

,geometry,label
0,"POLYGON ((85.80406 19.79352, 85.80406 19.79352...",1
1,"POLYGON ((85.80194 19.79276, 85.80195 19.79276...",1
2,"POLYGON ((85.80227 19.79280, 85.80227 19.79280...",1
3,"POLYGON ((85.80213 19.79275, 85.80214 19.79275...",1
4,"POLYGON ((85.80251 19.79264, 85.80251 19.79261...",1


In [23]:
bdt_gdf.groupby(['label']).agg(['count'])

,geometry
,count
label,
1,20742
2,145
3,1320
4,3
5,210
6,1
7,2526


# Save the Footprints

In [27]:
OUTPUT

'production/puri/modeloutput/rooftype'

In [29]:
print(f'Saving the footprints')
bdt_gdf.to_file(f'{OUTPUT}/{REGION}_footprints.geojson', driver='GeoJSON')

Saving the footprints
